In [2]:
install.packages("devtools")
install.packages("BiocManager")
BiocManager::install("GEOquery")
install.packages("remotes")
remotes::install_github("mkuhn/dict")
install.packages("sjmisc")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)
Installing package(s) 'GEOquery'
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'boot', 'brio', 'broom', 'caTools', 'class',
  'cli', 'cluster', 'codetools', 'colorspace', 'cpp11', 'crayon', 'crosstalk',
  'data.table', 'DBI', 'dbplyr', 'desc', 'diffobj', 'dplyr', 'DT', 'fansi',
  'farver', 'fastmap', 'flowCore', 'flowWorkspace', 'forcats', 'formatR',
  'ggplot2', 'git2r', 'gitcreds', 'glmnet', 'heatmaply', 'hexbin', 'hms',
  'htmltools', 'htmlwidgets', 'httpuv', 'ImmuneSpaceR', 'insight', 'ipred',
  'IRdisplay', 'IRkernel', 'isoband', 'jsonlite', 'knitr', 'lava', 'lifecycle',
  'limma', 'lubridate', 'magrittr', 'MASS', 'Matrix', 'matrixStats', 'memoise',
  'mgcv', 'mime', 'nlme', 'nnet', 'pbdZMQ', 'p

In [3]:
library(GEOquery)
library(data.table)
library(magrittr)
library(dplyr)
library(tidyr)
library(sjmisc)
library(dict)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


In [4]:
# read expsample to biosample
fn = "expsample_2_biosample.txt"
e2b = fread(fn, data.table = F)


# read biosample table
fn = "biosample.txt"
biosample = fread(fn, data.table = F)
biosample = biosample%>%select(SUBJECT_ACCESSION, BIOSAMPLE_ACCESSION, STUDY_ACCESSION)


# find gene expression result
fn = "expsample_public_repository.txt"
pub_repos = fread(fn, data.table = F)
pub_repos = pub_repos%>%filter(REPOSITORY_NAME=="GEO")


#(2) map gene expression result to biosample w/ expsample_accession
pub_repos1 = pub_repos%>%inner_join(e2b[, c("EXPSAMPLE_ACCESSION","BIOSAMPLE_ACCESSION")], by="EXPSAMPLE_ACCESSION")


#(3) map gene expression result to subject
pub_repos2 = pub_repos1%>%inner_join(biosample, by="BIOSAMPLE_ACCESSION")

In [5]:
head(pub_repos2)

,RESULT_ID,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME,BIOSAMPLE_ACCESSION,SUBJECT_ACCESSION,STUDY_ACCESSION
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,46870,ES1262321,GSM3584982,GEO,BS1028390,SUB205388,SDY1529
2,46871,ES1262322,GSM4041840,GEO,BS1028391,SUB205388,SDY1529
3,46872,ES1262323,GSM4041838,GEO,BS1028392,SUB205388,SDY1529
4,46873,ES1262324,GSM4041839,GEO,BS1028393,SUB205388,SDY1529
5,46874,ES1262325,GSM4041841,GEO,BS1028394,SUB205388,SDY1529
6,46875,ES1262326,GSM3584991,GEO,BS1028395,SUB205387,SDY1529


In [6]:
pub_repos2[pub_repos2$REPOSITORY_ACCESSION == 'GSM4041892', ]

,RESULT_ID,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME,BIOSAMPLE_ACCESSION,SUBJECT_ACCESSION,STUDY_ACCESSION
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
180,47049,ES1262500,GSM4041892,GEO,BS1028569,SUB205401,SDY1529


In [7]:
unique(pub_repos2$STUDY_ACCESSION)

[1] "SDY1529"

In [46]:
colnames(pub_repos2)

[1] "RESULT_ID"            "EXPSAMPLE_ACCESSION"  "REPOSITORY_ACCESSION"
[4] "REPOSITORY_NAME"      "BIOSAMPLE_ACCESSION"  "SUBJECT_ACCESSION"   
[7] "STUDY_ACCESSION"

In [74]:
gpl_dict = dict()

In [75]:
x0 = c('REPOSITORY_ACCESSION', 'BIOSAMPLE_ACCESSION', 'EXPSAMPLE_ACCESSION', 'SUBJECT_ACCESSION', 'GENE_VALUE_0')
df0 = data.frame(matrix(ncol = 6, nrow = 0))
colnames(df0) = x0

In [76]:
for(i in 1:length(pub_repos2$REPOSITORY_ACCESSION)) {
    ### LOADING TABLES
    GSM = getGEO(pub_repos2$REPOSITORY_ACCESSION[i], getGPL=FALSE)
    
    if(length(colnames(GSM@dataTable@table)) == 0)
        next
    
    genex = GSM@dataTable@table[,1:2]# it exists, load it
    platform_char = GSM@header$platform_id
  
    ### check if platform id already loaded
    if(platform_char %in% gpl_dict$keys()){
        ### at this point the platform table should only have cols ID and Symbol
        gene_prof = genex%>%inner_join(gpl_dict[[platform_char]], by=c("ID_REF"="ID"))
    
    } else { ### we have a unique platform GPL ID to add to the dict
        ### load unique GPL
        platform_obj = getGEO(platform_char, getGPL=FALSE)
        platform = platform_obj@dataTable@table
        
        if("ID" %in% colnames(platform) && str_contains(colnames(platform), "Symbol")){
            names(platform)[grepl("Symbol", names(platform))] = "Symbol"
            platform = platform[,c("ID","Symbol")]

            gpl_dict[[platform_char]] = platform ## add platform table to dict
            gene_prof = genex%>%inner_join(platform, by=c("ID_REF"="ID"))
      
        } else
            next
    }
      
    gene_profiles = gene_prof[ ,c("Symbol","VALUE")]
    gene_profiles = gene_profiles[!(gene_profiles$Symbol == "---" | gene_profiles$Symbol == ""), ]
    
    if(exists("gene_profiles")){
        add_row = data.frame(pub_repos2$REPOSITORY_ACCESSION[i], pub_repos2$BIOSAMPLE_ACCESSION[i], pub_repos2$EXPSAMPLE_ACCESSION[i], 
                             pub_repos2$SUBJECT_ACCESSION[i], gene_profiles[gene_profiles$Symbol == 'GZMH',][2])
        colnames(add_row) = x0
        df0 = rbind(df0, add_row)
    }
}

Using locally cached version of GSM3584982 found here:
/tmp/RtmpL2iCBG/GSM3584982.soft 
Using locally cached version of GPL10558 found here:
/tmp/RtmpL2iCBG/GPL10558.soft 
Warning message:
“3270 parsing failures.
  row        col           expected    actual         file
29537 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.571610 literal data
29538 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.545780 literal data
29539 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.554603 literal data
29540 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.437179 literal data
29541 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.128234 literal data
..... .......... .................. ......... ............
See problems(...) for more details.
”Using locally cached version of GSM4041840 found here:
/tmp/RtmpL2iCBG/GSM4041840.soft 
Using locally cached version of GSM4041838 found here:
/tmp/RtmpL2iCBG/GSM4041838.soft 
Using locally cached version of GSM4041839 found here:
/tmp/RtmpL2iCBG/GSM4041839.soft 
Using locally cached version of GSM4041841 found here:
/tmp/RtmpL2iCBG/GS

In [10]:
head(final_res1)

,Symbol,GSM4041892
,<chr>,<dbl>
1,ERCC-00162,6.510425
2,ERCC-00071,6.593000
3,ERCC-00009,6.538246
4,ERCC-00053,6.325730
5,ERCC-00144,6.358830
6,ERCC-00003,6.437396


In [64]:
head(df0)

,REPOSITORY_ACCESSION,BIOSAMPLE_ACCESSION,EXPSAMPLE_ACCESSION,SUBJECT_ACCESSION,SYMBOL,GENE_VALUE
,<fct>,<fct>,<fct>,<fct>,<chr>,<dbl>
15078,GSM3584982,BS1028390,ES1262321,SUB205388,GZMH,9.204821
14677,GSM4041840,BS1028391,ES1262322,SUB205388,GZMH,9.243751
146771,GSM4041838,BS1028392,ES1262323,SUB205388,GZMH,9.036877
146772,GSM4041839,BS1028393,ES1262324,SUB205388,GZMH,9.717368
146773,GSM4041841,BS1028394,ES1262325,SUB205388,GZMH,9.448006
150781,GSM3584991,BS1028395,ES1262326,SUB205387,GZMH,9.045475


In [11]:
dim(final_res1)#****************

[1] 44053     2

In [12]:
#unique(final_res1$Symbol)[93:200]

In [13]:
final_res1[final_res1$Symbol == 'GZMH',]

,Symbol,GSM4041892
,<chr>,<dbl>
14677,GZMH,10.58033


In [14]:
fn = "neut_ab_titer_result.txt"
titer = fread(fn, data.table = F)

In [15]:
head(titer)

,RESULT_ID,ARM_ACCESSION,BIOSAMPLE_ACCESSION,COMMENTS,EXPERIMENT_ACCESSION,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME,STUDY_ACCESSION,STUDY_TIME_COLLECTED,STUDY_TIME_COLLECTED_UNIT,SUBJECT_ACCESSION,UNIT_PREFERRED,UNIT_REPORTED,VALUE_PREFERRED,VALUE_REPORTED,VIRUS_STRAIN_PREFERRED,VIRUS_STRAIN_REPORTED,WORKSPACE_ID
,<int>,<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<lgl>,<chr>,<int>
1,797961,ARM5007,BS1028555,NA,EXP28324,ES1262501,NA,NA,SDY1529,0,Days,SUB205386,Antibody titer,Antibody titer,80,80,NA,YF-17D,6133
2,797962,ARM5007,BS1028395,NA,EXP28324,ES1262502,NA,NA,SDY1529,0,Days,SUB205387,Antibody titer,Antibody titer,80,80,NA,YF-17D,6133
3,797963,ARM5007,BS1028390,NA,EXP28324,ES1262503,NA,NA,SDY1529,0,Days,SUB205388,Antibody titer,Antibody titer,80,80,NA,YF-17D,6133
4,797964,ARM5007,BS1028540,NA,EXP28324,ES1262504,NA,NA,SDY1529,0,Days,SUB205389,Antibody titer,Antibody titer,80,80,NA,YF-17D,6133
5,797965,ARM5007,BS1028445,NA,EXP28324,ES1262505,NA,NA,SDY1529,0,Days,SUB205390,Antibody titer,Antibody titer,80,80,NA,YF-17D,6133
6,797966,ARM5007,BS1028415,NA,EXP28324,ES1262506,NA,NA,SDY1529,0,Days,SUB205391,Antibody titer,Antibody titer,80,80,NA,YF-17D,6133


In [16]:
titer1 = titer[, c("BIOSAMPLE_ACCESSION", "EXPSAMPLE_ACCESSION", "SUBJECT_ACCESSION", "STUDY_TIME_COLLECTED", "VALUE_PREFERRED")]

In [17]:
head(pub_repos2)

,RESULT_ID,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME,BIOSAMPLE_ACCESSION,SUBJECT_ACCESSION,STUDY_ACCESSION
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,46870,ES1262321,GSM3584982,GEO,BS1028390,SUB205388,SDY1529
2,46871,ES1262322,GSM4041840,GEO,BS1028391,SUB205388,SDY1529
3,46872,ES1262323,GSM4041838,GEO,BS1028392,SUB205388,SDY1529
4,46873,ES1262324,GSM4041839,GEO,BS1028393,SUB205388,SDY1529
5,46874,ES1262325,GSM4041841,GEO,BS1028394,SUB205388,SDY1529
6,46875,ES1262326,GSM3584991,GEO,BS1028395,SUB205387,SDY1529


In [18]:
repos_titer = titer1%>%inner_join(pub_repos2[,c("EXPSAMPLE_ACCESSION", "REPOSITORY_ACCESSION", "BIOSAMPLE_ACCESSION", "SUBJECT_ACCESSION")], by=c("BIOSAMPLE_ACCESSION", "SUBJECT_ACCESSION"))

In [19]:
head(repos_titer)

,BIOSAMPLE_ACCESSION,EXPSAMPLE_ACCESSION.x,SUBJECT_ACCESSION,STUDY_TIME_COLLECTED,VALUE_PREFERRED,EXPSAMPLE_ACCESSION.y,REPOSITORY_ACCESSION
,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,BS1028555,ES1262501,SUB205386,0,80,ES1262486,GSM3584988
2,BS1028395,ES1262502,SUB205387,0,80,ES1262326,GSM3584991
3,BS1028390,ES1262503,SUB205388,0,80,ES1262321,GSM3584982
4,BS1028540,ES1262504,SUB205389,0,80,ES1262471,GSM3584985
5,BS1028445,ES1262505,SUB205390,0,80,ES1262376,GSM3584983
6,BS1028415,ES1262506,SUB205391,0,80,ES1262346,GSM3584995


In [49]:
colnames(repos_titer)

[1] "BIOSAMPLE_ACCESSION"   "EXPSAMPLE_ACCESSION.x" "SUBJECT_ACCESSION"    
[4] "STUDY_TIME_COLLECTED"  "VALUE_PREFERRED"       "EXPSAMPLE_ACCESSION.y"
[7] "REPOSITORY_ACCESSION"

In [20]:
length(repos_titer$REPOSITORY_ACCESSION)

[1] 72

In [23]:
#gs = getGEO('GSM4041971', getGPL=FALSE)

In [24]:
#gex = gs@dataTable@table[,1:2]

In [30]:
#gex[gex$ID_REF == "GZMH", ]

In [29]:
#platform_char = gs@header$platform_id

In [55]:
#platform_obj = getGEO(platform_char, getGPL=FALSE)
#platform = platform_obj@dataTable@table

In [31]:
#names(platform)[grepl("Symbol", names(platform))] = "Symbol"
#platform = platform[,c("ID","Symbol")]

#gen_pro= gex%>%inner_join(platform, by=c("ID_REF"="ID"))      

In [32]:
#head(gen_pro)

,ID_REF,VALUE,Symbol
,<chr>,<dbl>,<chr>
1,ILMN_3166687,6.446268,ERCC-00162
2,ILMN_3165565,6.446268,ERCC-00071
3,ILMN_3164808,6.461869,ERCC-00009
4,ILMN_3165363,6.349310,ERCC-00053
5,ILMN_3166504,6.413672,ERCC-00144
6,ILMN_3164750,6.430380,ERCC-00003


In [33]:
#gene_profs = gen_pro[ ,c("Symbol","VALUE")]
#gene_profs = gene_profs[!(gene_profs$Symbol == "---" | gene_profs$Symbol == ""), ]

In [34]:
#head(gene_profs)

,Symbol,VALUE
,<chr>,<dbl>
1,ERCC-00162,6.446268
2,ERCC-00071,6.446268
3,ERCC-00009,6.461869
4,ERCC-00053,6.349310
5,ERCC-00144,6.413672
6,ERCC-00003,6.430380


In [44]:
#gene_profs[gene_profs$Symbol == 'GZMH',]#[1], [2]

,Symbol,VALUE
,<chr>,<dbl>
14677,GZMH,9.713595


In [77]:
gpl_dict = dict()

In [78]:
x = c('REPOSITORY_ACCESSION', 'BIOSAMPLE_ACCESSION', 'EXPSAMPLE_ACCESSION', 
      'SUBJECT_ACCESSION', 'STUDY_TIME_COLLECTED', 'GENE_VALUE', 'VALUE_PREFERRED')
df = data.frame(matrix(ncol = 9, nrow = 0))
colnames(df) = x

In [79]:
for(i in 1:length(repos_titer$REPOSITORY_ACCESSION)) {
    ### LOADING TABLES
    GSM = getGEO(repos_titer$REPOSITORY_ACCESSION[i], getGPL=FALSE)
    
    if(length(colnames(GSM@dataTable@table)) == 0)
        next
    
    genex = GSM@dataTable@table[,1:2]# it exists, load it
    platform_char = GSM@header$platform_id
  
    ### check if platform id already loaded
    if(platform_char %in% gpl_dict$keys()){
        ### at this point the platform table should only have cols ID and Symbol
        gene_prof = genex%>%inner_join(gpl_dict[[platform_char]], by=c("ID_REF"="ID"))
    
    } else { ### we have a unique platform GPL ID to add to the dict
        ### load unique GPL
        platform_obj = getGEO(platform_char, getGPL=FALSE)
        platform = platform_obj@dataTable@table
        
        if("ID" %in% colnames(platform) && str_contains(colnames(platform), "Symbol")){
            names(platform)[grepl("Symbol", names(platform))] = "Symbol"
            platform = platform[,c("ID","Symbol")]

            gpl_dict[[platform_char]] = platform ## add platform table to dict
            gene_prof = genex%>%inner_join(platform, by=c("ID_REF"="ID"))
      
        } else
            next
    }
      
    gene_profiles = gene_prof[ ,c("Symbol","VALUE")]
    gene_profiles = gene_profiles[!(gene_profiles$Symbol == "---" | gene_profiles$Symbol == ""), ]
    
    if(exists("gene_profiles")){
        add_row = data.frame(repos_titer$REPOSITORY_ACCESSION[i], repos_titer$BIOSAMPLE_ACCESSION[i], repos_titer$EXPSAMPLE_ACCESSION.x[i],#repos_titer$EXPSAMPLE_ACCESSION.y[i]
                             repos_titer$SUBJECT_ACCESSION[i], repos_titer$STUDY_TIME_COLLECTED[i], gene_profiles[gene_profiles$Symbol == 'GZMH',][2], 
                             repos_titer$VALUE_PREFERRED[i])
        colnames(add_row) = x
        df = rbind(df, add_row)
    }
}

Using locally cached version of GSM3584988 found here:
/tmp/RtmpL2iCBG/GSM3584988.soft 
Using locally cached version of GPL10558 found here:
/tmp/RtmpL2iCBG/GPL10558.soft 
Warning message:
“3270 parsing failures.
  row        col           expected    actual         file
29537 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.571610 literal data
29538 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.545780 literal data
29539 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.554603 literal data
29540 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.437179 literal data
29541 Unigene_ID 1/0/T/F/TRUE/FALSE Hs.128234 literal data
..... .......... .................. ......... ............
See problems(...) for more details.
”Using locally cached version of GSM3584991 found here:
/tmp/RtmpL2iCBG/GSM3584991.soft 
Using locally cached version of GSM3584982 found here:
/tmp/RtmpL2iCBG/GSM3584982.soft 
Using locally cached version of GSM3584985 found here:
/tmp/RtmpL2iCBG/GSM3584985.soft 
Using locally cached version of GSM3584983 found here:
/tmp/RtmpL2iCBG/GS

In [80]:
head(df)# SPEARMAN CORR, P-VAL -- cor.test()

,REPOSITORY_ACCESSION,BIOSAMPLE_ACCESSION,EXPSAMPLE_ACCESSION,SUBJECT_ACCESSION,STUDY_TIME_COLLECTED,GENE_VALUE,VALUE_PREFERRED
,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<int>
15078,GSM3584988,BS1028555,ES1262501,SUB205386,0,10.095504,80
150781,GSM3584991,BS1028395,ES1262502,SUB205387,0,9.045475,80
150782,GSM3584982,BS1028390,ES1262503,SUB205388,0,9.204821,80
150783,GSM3584985,BS1028540,ES1262504,SUB205389,0,10.462735,80
150784,GSM3584983,BS1028445,ES1262505,SUB205390,0,8.739116,80
150785,GSM3584995,BS1028415,ES1262506,SUB205391,0,9.775270,80


In [120]:
study0 = df[df$STUDY_TIME_COLLECTED == 0, 'GENE_VALUE']
study0

[1] 10.095504  9.045475  9.204821 10.462735  8.739116  9.775270  8.948616
 [8]  9.049519  9.347965 10.253440  9.746502 10.804556 10.517990 10.031448
[15] 10.157734  9.981411  8.964735  9.455588  9.571882  9.386009  9.211446
[22]  9.014174  9.443932 10.243976 10.824085 10.202156 10.133653  9.517990
[29]  9.312797  9.608059  9.012138  9.834339  9.367878  9.521630  9.230953
[36]  9.771583

In [121]:
study84 = df[df$STUDY_TIME_COLLECTED == 84, 'GENE_VALUE']

In [112]:
#install.packages("ggpubr")
if(!require(devtools)) install.packages("devtools")
devtools::install_github("kassambara/ggpubr")

Loading required package: devtools
Loading required package: usethis


vctrs        (0.3.4      -> 0.3.6     ) [CRAN]
utf8         (1.1.4      -> 1.2.1     ) [CRAN]
rlang        (0.4.8      -> 0.4.10    ) [CRAN]
lifecycle    (0.2.0      -> 1.0.0     ) [CRAN]
fansi        (0.4.1      -> 0.4.2     ) [CRAN]
crayon       (1.3.4      -> 1.4.1     ) [CRAN]
cli          (2.1.0      -> 2.3.1     ) [CRAN]
cpp11        (0.2.3      -> 0.2.6     ) [CRAN]
BH           (1.72.0-3   -> 1.75.0-0  ) [CRAN]
mime         (0.9        -> 0.10      ) [CRAN]
pillar       (1.4.6      -> 1.5.1     ) [CRAN]
xfun         (0.19       -> 0.22      ) [CRAN]
hms          (0.5.3      -> 1.0.0     ) [CRAN]
forcats      (0.5.0      -> 0.5.1     ) [CRAN]
RcppArmad... (0.10.1.2.0 -> 0.10.2.2.0) [CRAN]
matrixStats  (0.57.0     -> 0.58.0    ) [CRAN]
tidyr        (1.1.2      -> 1.1.3     ) [CRAN]
backports    (1.1.10     -> 1.2.1     ) [CRAN]
tibble       (3.0.4      -> 3.1.0     ) [CRAN]
data.table   (1.13.2     -> 1.14.0    ) [CRAN]
knitr        (1.30       -> 1.31      ) [CRAN]
magrittr     

Skipping 1 packages not available: foreign
Installing 35 packages: vctrs, utf8, rlang, lifecycle, fansi, crayon, cli, cpp11, BH, mime, pillar, xfun, hms, forcats, RcppArmadillo, matrixStats, tidyr, backports, tibble, data.table, knitr, magrittr, dplyr, Rcpp, lme4, rio, pbkrtest, colorspace, farver, withr, isoband, car, rstatix, cowplot, ggrepel
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


✔  checking for file ‘/tmp/RtmpL2iCBG/remotese777049613c/kassambara-ggpubr-ac5a01f/DESCRIPTION’ ...
─  preparing ‘ggpubr’:
✔  checking DESCRIPTION meta-information ...
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘ggpubr_0.4.0.999.tar.gz’
   


In [119]:
#library("ggpubr")

In [122]:
cor.test(study0, study84, method="spearman")#statistically significant vars


	Spearman's rank correlation rho

data:  study0 and study84
S = 2190, p-value = 2.104e-06
alternative hypothesis: true rho is not equal to 0
sample estimates:
      rho 
0.7181467 


In [105]:
df0[df0$REPOSITORY_ACCESSION == "GSM3584988", ]

,REPOSITORY_ACCESSION,BIOSAMPLE_ACCESSION,EXPSAMPLE_ACCESSION,SUBJECT_ACCESSION,GENE_VALUE_0
,<fct>,<fct>,<fct>,<fct>,<dbl>
1507833,GSM3584988,BS1028555,ES1262486,SUB205386,10.0955


In [102]:
table(df$VALUE_PREFERRED)


   80   160   320   640  1280  2560  5120 10240 
   35     2     2     6    10    10     6     1 

In [81]:
head(df0)

,REPOSITORY_ACCESSION,BIOSAMPLE_ACCESSION,EXPSAMPLE_ACCESSION,SUBJECT_ACCESSION,GENE_VALUE_0
,<fct>,<fct>,<fct>,<fct>,<dbl>
15078,GSM3584982,BS1028390,ES1262321,SUB205388,9.204821
14677,GSM4041840,BS1028391,ES1262322,SUB205388,9.243751
146771,GSM4041838,BS1028392,ES1262323,SUB205388,9.036877
146772,GSM4041839,BS1028393,ES1262324,SUB205388,9.717368
146773,GSM4041841,BS1028394,ES1262325,SUB205388,9.448006
150781,GSM3584991,BS1028395,ES1262326,SUB205387,9.045475


In [26]:
fin_res1[fin_res1$Symbol == 'GZMH',]

,Symbol,GSM4041971
,<chr>,<dbl>
14677,GZMH,9.713595


In [58]:
df[df$gsm_col == 'GSM4041971', ]# WANT TO ADD COL VALUES 

,gsm_col,study_time_col
,<fct>,<int>
72,GSM4041971,84


In [ ]:
#if(exists('final_res1')){
#    fwrite(final_res1, paste0(i,"_" ,Sys.time(),"_" ,".csv"))
#}

#fwrite(data.frame(gpl_vec, gsm_vec), paste0("nonfunctional_gpl_gsm_dataframe" ,".csv"))# non func gpl_vec, gsm_vec
#fwrite(data.frame(nonfunc_gsm), paste0("empty_gsm_data_table" ,".csv"))#empty gsm data table